In [13]:
%matplotlib inline
from effective_quadratures.PolyParams import PolynomialParam
from effective_quadratures.PolyParentFile import PolyParent
from effective_quadratures.IndexSets import IndexSet
import effective_quadratures.ComputeStats as stats
from effective_quadratures.EffectiveQuadSubsampling import EffectiveSubsampling
import numpy as np
import matplotlib.pyplot as plt

<h1> The Namesake: Effective Quadrature Subsampling

In this notebook we demonstrate what effective quadrature subsampling is. We begin by defining a simple two-dimensional function
$$f(x_1, x_2) = exp(x_1 + x_2)$$
where $x_1$ and $x_2$ are uniformly distributed over the cube $x_1, x_2 
\in [-1,1]$. 

Lets first understand what exactly effective quadrature subsampling is and how it computes the pseudospectral coefficients. [Insert Explanation!]

<img src='Matrices.png'>

In [14]:
def exponential_function(x):
    return np.exp(x[0] + x[1])

Declare the two inputs and their ranges

In [15]:
no_pts_x1 = 5
no_pts_x2 = 5
min_range = -1.0
max_range = 1.0
x1 = PolynomialParam("Uniform", min_range, max_range, [], [], [], no_pts_x1)
x2 = PolynomialParam("Uniform", min_range, max_range, [], [], [], no_pts_x2)
x1x2 = []
x1x2.append(x1)
x1x2.append(x2)

We will opt for a hyperbolic cross basis, though one can also opt for a total order basis. Bear in mind that we are trying to escape the cost associated with a more conventional tensor grid or sparse grid basis. A hyperbolic cross takes in a *q* parameter. When *q=1* we have a total order basis. 

In [16]:
q = 0.5
hyperbolic_cross = IndexSet("hyperbolic cross", [no_pts_x1, no_pts_x2], q)
maximum_number_of_evals = IndexSet.getCardinality(hyperbolic_cross)

The class Effective_Subsampling has holds all the function calls required for effective quadrature subsampling. We begin by declaring an EffectiveSubsampling object that takes as inputs the uncertain parameters, the index set and a derivative flag. In this notebook as derivatives are not considered the derivative flag is set to 0.

In [26]:
effectiveQuads = EffectiveSubsampling(x1x2, hyperbolic_cross, 0)
A_tall, pts = EffectiveSubsampling.getAsubsampled(effectiveQuads, maximum_number_of_evals)

---Square Matrix-----
[ 0  4 20 24  8 16 12 19 13 10]


[[  5.00000000e-01  -7.84774767e-01   8.18113399e-01  -6.62801939e-01
    3.68603189e-01  -7.84774767e-01   1.23174287e+00   8.18113399e-01
   -6.62801939e-01   3.68603189e-01]
 [  5.00000000e-01   7.84774767e-01   8.18113399e-01   6.62801939e-01
    3.68603189e-01  -7.84774767e-01  -1.23174287e+00   8.18113399e-01
   -6.62801939e-01   3.68603189e-01]
 [  5.00000000e-01  -7.84774767e-01   8.18113399e-01  -6.62801939e-01
    3.68603189e-01   7.84774767e-01  -1.23174287e+00   8.18113399e-01
    6.62801939e-01   3.68603189e-01]
 [  5.00000000e-01   7.84774767e-01   8.18113399e-01   6.62801939e-01
    3.68603189e-01   7.84774767e-01   1.23174287e+00   8.18113399e-01
    6.62801939e-01   3.68603189e-01]
 [  5.00000000e-01   4.66328102e-01  -7.27574069e-02  -5.52144624e-01
   -5.16751337e-01  -4.66328102e-01  -4.34923797e-01  -7.27574069e-02
    5.52144624e-01  -5.16751337e-01]
 [  5.00000000e-01  -4.66328102e-01  -7.27574069e-02   5.52